In [ ]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep

# Set up the Chrome driver
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
driver = webdriver.Chrome('./chromedriver', options=options)
wait = WebDriverWait(driver, 10)


url = 'https://www.giustizia-amministrativa.it/web/guest/dcsnprr?p_p_id=GaSearch_INSTANCE_2NDgCF3zWBwk&p_p_state=normal&p_p_mode=view&_GaSearch_INSTANCE_2NDgCF3zWBwk_javax.portlet.action=searchProvvedimenti&p_auth=mtYPa33V&p_p_lifecycle=0'
driver.get(url)

text_name = '_GaSearch_INSTANCE_2NDgCF3zWBwk_searchtextProvvedimenti'
input_field = driver.find_element("name",text_name)

input_field.send_keys("sentenza")




dropdown_element = driver.find_element("name", '_GaSearch_INSTANCE_2NDgCF3zWBwk_pageResultsProvvedimenti')
dropdown = Select(dropdown_element)
dropdown.select_by_value("60")

input_field.send_keys(Keys.ENTER)


#wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="col-sm-9 risultati--ricerca"]')))
wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "ricerca--item")))
#sleep(10)
content = driver.page_source
nextPage(driver)
#driver.quit()

In [7]:
def getPageElement(page_content):
    page = BeautifulSoup(page_content, 'html.parser')
    footer = page.find_all('article')
    text = footer[60].find('div',{'class':'col-sm-12'}).text
    text = text.split('Risultati da ')[1].split(' ')
    print(text[2],text[4])
    return text[2],text[4]

#getPageElement(content)

In [27]:
driver.quit()

In [79]:
from bs4 import BeautifulSoup

content = driver.page_source

# Get the page source and parse it with BeautifulSoup
#content = driver.page_source
soup = BeautifulSoup(content, 'html.parser')

articles = soup.find_all('article')
# Extract the data you want
print(len(articles))


61


In [86]:
#print(articles[0])
prefix = 'https://www.giustizia-amministrativa.it'

PDF = []
ID = []
for article in articles:
    linkToPDF = article.find('a', {'class': 'visited-provvedimenti', 'title': 'Versione highlighted'})
    if(linkToPDF == None):
        continue
    PDFurl = prefix + linkToPDF['href']
    ID.append(linkToPDF.text)
    PDF.append(PDFurl)

print(len(PDF))
print(len(ID))
print(ID)

print(getPDFs([content])[1][0])

60
60
['202306351 (ROMA, SEZIONE 2B) html', '202306345 (ROMA, SEZIONE 1B) html', '202306344 (ROMA, SEZIONE 5B) html', '202306342 (ROMA, SEZIONE 2T) html', '202306341 (ROMA, SEZIONE 2T) html', '202306340 (ROMA, SEZIONE 2T) html', '202306339 (ROMA, SEZIONE 2T) html', '202306338 (ROMA, SEZIONE 2T) html', '202306337 (ROMA, SEZIONE 2T) html', '202306336 (ROMA, SEZIONE 2T) html', '202306334 (ROMA, SEZIONE 2T) html', '202306333 (ROMA, SEZIONE 2T) html', '202306332 (ROMA, SEZIONE 2T) html', '202306331 (ROMA, SEZIONE 3T) html', '202306330 (ROMA, SEZIONE 2S) html', '202306329 (ROMA, SEZIONE 2) html', '202306328 (ROMA, SEZIONE 2S) html', '202306327 (ROMA, SEZIONE 2S) html', '202306325 (ROMA, SEZIONE 2) html', '202306324 (ROMA, SEZIONE 2) html', '202306323 (ROMA, SEZIONE 2) html', '202306322 (ROMA, SEZIONE 2) html', '202306321 (ROMA, SEZIONE 2S) html', '202306319 (ROMA, SEZIONE 2S) html', '202306318 (ROMA, SEZIONE 1Q) html', '202306315 (ROMA, SEZIONE 2) html', '202306314 (ROMA, SEZIONE 2) html', '

In [48]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup

batch_size = 50 #Has to be a multiple by 60

def scrapeData(earlyStopping = False):
    pages = []
    # Set up the Chrome driver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome('./chromedriver', options=options)
    wait = WebDriverWait(driver, 10)
    
    #Set the url
    url = 'https://www.giustizia-amministrativa.it/web/guest/dcsnprr?p_p_id=GaSearch_INSTANCE_2NDgCF3zWBwk&p_p_state=normal&p_p_mode=view&_GaSearch_INSTANCE_2NDgCF3zWBwk_javax.portlet.action=searchProvvedimenti&p_auth=mtYPa33V&p_p_lifecycle=0'
    driver.get(url)

    #Set the query correctly
    text_name = '_GaSearch_INSTANCE_2NDgCF3zWBwk_searchtextProvvedimenti'
    input_field = driver.find_element("name",text_name)

    input_field.send_keys("sentenza")

    dropdown_element = driver.find_element("name", '_GaSearch_INSTANCE_2NDgCF3zWBwk_pageResultsProvvedimenti')
    dropdown = Select(dropdown_element)
    dropdown.select_by_value("60")

    #Press enter to start the search
    input_field.send_keys(Keys.ENTER)


    #Extract the page
    page = extractPage(wait,driver)
    pages.append(page)
    current,total = getPageElement(page)

    cycle = 1
    
    #Extract the other pages 
    if(earlyStopping != False):
        total = earlyStopping
    while(int(current) < int(total)):
        nextPage(driver)
        page = extractPage(wait,driver)
        pages.append(page)
        current,total = getPageElement(page)
        if(earlyStopping != False):
            total = earlyStopping

    driver.quit()

    return pages

def nextPage(dr):
    nextPageButton = dr.find_element(By.LINK_TEXT, 'Successivo')
    nextPageButton.click()
    #print(nextPageButton)


def extractPage(w,d):
    #wait.until(EC.visibility_of_element_located((By.XPATH, '//div[@class="col-sm-9 risultati--ricerca"]')))
    w.until(EC.visibility_of_element_located((By.CLASS_NAME, "ricerca--item")))
    #sleep(10)
    content = d.page_source
    return content


def saveBatch(listOfPages,id):
    batchPDF,batchID = getPDFs(listOfPages)
    #Create a Panda dataframe that contains the batch of PDFs and their IDs
    #Create a list of list containing pairs of PDF and ID
    data = list(zip(batchPDF,batchID))

    #convert data to dataframe
    df = pd.DataFrame(data,columns=['PDF_link','ID'])
    #print(df)
    #save the dataframe to a csv file
    df.to_csv('./dataset/batch'+str(id)+'.csv',index=True,header=True)

def scrapeBatch():
    pages = []
    # Set up the Chrome driver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome('./chromedriver', options=options)
    wait = WebDriverWait(driver, 10)
    
    #Set the url
    url = 'https://www.giustizia-amministrativa.it/web/guest/dcsnprr?p_p_id=GaSearch_INSTANCE_2NDgCF3zWBwk&p_p_state=normal&p_p_mode=view&_GaSearch_INSTANCE_2NDgCF3zWBwk_javax.portlet.action=searchProvvedimenti&p_auth=mtYPa33V&p_p_lifecycle=0'
    driver.get(url)

    #Set the query correctly
    text_name = '_GaSearch_INSTANCE_2NDgCF3zWBwk_searchtextProvvedimenti'
    input_field = driver.find_element("name",text_name)

    input_field.send_keys("sentenza")

    dropdown_element = driver.find_element("name", '_GaSearch_INSTANCE_2NDgCF3zWBwk_pageResultsProvvedimenti')
    dropdown = Select(dropdown_element)
    dropdown.select_by_value("60")

    #Press enter to start the search
    input_field.send_keys(Keys.ENTER)


    #Extract the page
    page = extractPage(wait,driver)
    pages.append(page)
    current,total = getPageElement(page)

    batch_id = 0
    #Extract the other pages 
    while(int(current) < int(total)):
        nextPage(driver)
        page = extractPage(wait,driver)
        pages.append(page)
        current,total = getPageElement(page)
        if(len(pages) % batch_size == 0):
            print(f"Saving batch of {len(pages)*60} pages")
            saveBatch(pages,batch_id)
            batch_id += 1
            pages = []
            #Give a break to the server
            sleep(60)
            break
        print(f'Cycles:{batch_id}, {len(pages)}')

    driver.quit()

    return pages

In [42]:
def getPDFs(pages):
    PDF = []
    ID = []

    for page in pages:
        listOfItems = BeautifulSoup(page, 'html.parser')
        articles = listOfItems.find_all('article')
        prefix = 'https://www.giustizia-amministrativa.it'
        
        for article in articles:
            linkToPDF = article.find('a', {'class': 'visited-provvedimenti', 'title': 'Versione highlighted'})
            if(linkToPDF == None):
                continue
            PDFurl = prefix + linkToPDF['href']
            ID.append(linkToPDF.text)
            PDF.append(PDFurl)

    print(f'Extracted {len(PDF)} PDFs and {len(ID)} descriptions')
    return PDF,ID

In [49]:

pagine = scrapeBatch()
#print(f'Estratte {len(pagine)} pagine da 60 sentenze!')

#PDFs,IDs = getPDFs(pagine)

/var/folders/l8/hdp9mfm55nq2xqyctskbfdvr0000gn/T/ipykernel_2730/3102292554.py:92: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver', options=options)


60 1339271
120 1339271
Cycles:0, 2
180 1339271
Cycles:0, 3
240 1339271
Cycles:0, 4
300 1339271
Cycles:0, 5
360 1339271
Cycles:0, 6
420 1339271
Cycles:0, 7
480 1339271
Cycles:0, 8
540 1339271
Cycles:0, 9
600 1339271
Cycles:0, 10
660 1339271
Cycles:0, 11
720 1339271
Cycles:0, 12
780 1339271
Cycles:0, 13
840 1339271
Cycles:0, 14
900 1339271
Cycles:0, 15
960 1339271
Cycles:0, 16
1020 1339271
Cycles:0, 17
1080 1339271
Cycles:0, 18
1140 1339271
Cycles:0, 19
1200 1339271
Cycles:0, 20
1260 1339271
Cycles:0, 21
1320 1339271
Cycles:0, 22
1380 1339271
Cycles:0, 23
1440 1339271
Cycles:0, 24
1500 1339271
Cycles:0, 25
1560 1339271
Cycles:0, 26
1620 1339271
Cycles:0, 27
1680 1339271
Cycles:0, 28
1740 1339271
Cycles:0, 29
1800 1339271
Cycles:0, 30
1860 1339271
Cycles:0, 31
1920 1339271
Cycles:0, 32
1980 1339271
Cycles:0, 33
2040 1339271
Cycles:0, 34
2100 1339271
Cycles:0, 35
2160 1339271
Cycles:0, 36
2220 1339271
Cycles:0, 37
2280 1339271
Cycles:0, 38
2340 1339271
Cycles:0, 39
2400 1339271
Cycles:0, 4

In [28]:
print(IDs)
print(PDFs[2])



['202306392 (ROMA, SEZIONE 2Q) html', '202306391 (ROMA, SEZIONE 2T) html', '202306390 (ROMA, SEZIONE 1Q) html', '202306389 (ROMA, SEZIONE 2Q) html', '202306388 (ROMA, SEZIONE 4B) html', '202306387 (ROMA, SEZIONE 1Q) html', '202306386 (ROMA, SEZIONE 4B) html', '202306384 (ROMA, SEZIONE 1Q) html', '202306383 (ROMA, SEZIONE 2B) html', '202306382 (ROMA, SEZIONE 2B) html', '202306381 (ROMA, SEZIONE 5B) html', '202306380 (ROMA, SEZIONE 5B) html', '202306379 (ROMA, SEZIONE 2B) html', '202306378 (ROMA, SEZIONE 2B) html', '202306377 (ROMA, SEZIONE 2B) html', '202306376 (ROMA, SEZIONE 2B) html', '202306375 (ROMA, SEZIONE 4B) html', '202306374 (ROMA, SEZIONE 1B) html', '202306373 (ROMA, SEZIONE 3Q) html', '202306372 (ROMA, SEZIONE 4B) html', '202306371 (ROMA, SEZIONE 2B) html', '202306370 (ROMA, SEZIONE 4B) html', '202306369 (ROMA, SEZIONE 1T) html', '202306368 (ROMA, SEZIONE 2B) html', '202306367 (ROMA, SEZIONE 2B) html', '202306366 (ROMA, SEZIONE 2B) html', '202306365 (ROMA, SEZIONE 1Q) html', 